In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as plt 
%matplotlib inline
plt.rc("font", size=14)
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
import seaborn as sns

from sklearn import metrics
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import log_loss, f1_score
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import RFE

from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout

sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)

Using TensorFlow backend.


Load data from csv file

In [2]:
data = pd.read_csv('DR_Demo_Lending_Club.csv', header=0)

In [3]:
data.describe()

,Id,is_bad,annual_inc,debt_to_income,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,collections_12_mths_ex_med,mths_since_last_major_derog
count,10000.00000,10000.000000,9999.000000,10000.000000,9995.000000,9995.000000,3684.000000,840.000000,9995.000000,9995.000000,1.000000e+04,9974.000000,9995.000000,9968.0,10000.000000
mean,5000.50000,0.129500,68203.011539,13.338704,0.148174,1.066933,35.890337,61.652381,9.334567,0.060130,1.427101e+04,48.450771,22.011306,0.0,2.002200
std,2886.89568,0.335769,48590.252757,6.754212,0.506270,1.476052,22.361443,46.189619,4.526590,0.251245,2.543791e+04,28.220557,11.709400,0.0,0.810963
min,1.00000,0.000000,2000.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000e+00,0.000000,1.000000,0.0,1.000000
25%,2500.75000,0.000000,40000.000000,8.160000,0.000000,0.000000,18.000000,0.000000,6.000000,0.000000,3.524500e+03,25.000000,13.000000,0.0,1.000000
50%,5000.50000,0.000000,58000.000000,13.410000,0.000000,1.000000,34.000000,86.000000,9.000000,0.000000,8.645500e+03,48.700000,20.000000,0.0,2.000000
75%,7500.25000,0.000000,82000.000000,18.692500,0.000000,2.000000,53.000000,101.000000,12.000000,0.000000,1.695225e+04,71.800000,29.000000,0.0,3.000000
max,10000.00000,1.000000,900000.000000,29.990000,11.000000,25.000000,120.000000,119.000000,39.000000,3.000000,1.207359e+06,100.600000,90.000000,0.0,3.000000


In [4]:
data.head()

,Id,is_bad,emp_title,emp_length,home_ownership,annual_inc,verification_status,pymnt_plan,Notes,purpose_cat,...,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code
0,1,0,Time Warner Cable,10,MORTGAGE,50000.0,not verified,n,NaN,medical,...,NaN,15.0,0.0,12087,12.1,44.0,f,0.0,1,PC4
1,2,0,Ottawa University,1,RENT,39216.0,not verified,n,Borrower added on 04/14/11 > I will be using...,debt consolidation,...,NaN,4.0,0.0,10114,64.0,5.0,f,0.0,2,PC1
2,3,0,Kennedy Wilson,4,RENT,65000.0,not verified,n,NaN,credit card,...,NaN,4.0,0.0,81,0.6,8.0,f,0.0,3,PC4
3,4,0,TOWN OF PLATTEKILL,10,MORTGAGE,57500.0,not verified,n,NaN,debt consolidation,...,NaN,6.0,0.0,10030,37.1,23.0,f,0.0,2,PC2
4,5,0,Belmont Correctional,10,MORTGAGE,50004.0,VERIFIED - income,n,"I want to consolidate my debt, pay for a vacat...",debt consolidation,...,NaN,8.0,0.0,10740,40.4,21.0,f,0.0,3,PC3


In [5]:
count_bad = len(data[data['is_bad']==1])
count_good = len(data[data['is_bad']==0])
pct_of_bad = count_bad/(count_bad+count_good)
print("percentage of bad loan is ", pct_of_bad*100)
pct_of_good = count_good/(count_bad+count_good)
print("percentage of good loan is ", pct_of_good*100)

percentage of bad loan is  12.950000000000001
percentage of good loan is  87.05000000000001


The data are completely out of balance. The learning algorithms will be very bias. I am not sure rather or not I should remove some data to make their about 50%/50% for each binary classification.

In [6]:
# temporary added cells

data = data.drop('purpose', axis = 1) 
data = data.drop('Id', axis = 1) 
#data = data.drop('emp_title', axis = 1)
data = data.drop('Notes', axis = 1)

data['earliest_cr_line'].fillna(data['earliest_cr_line'].value_counts().index[0], inplace=True)
data['policy_code'].fillna(data['policy_code'].value_counts().index[0], inplace=True)
data['emp_title'].fillna(data['emp_title'].value_counts().index[0], inplace=True)
data['annual_inc'].fillna(data['annual_inc'].median(), inplace=True)
data['open_acc'].fillna(data['open_acc'].median(), inplace=True)

In [7]:
for i in data:
    if data[i].isna().values.any():
        print(i, "There is NULL Values")

delinq_2yrs There is NULL Values
inq_last_6mths There is NULL Values
mths_since_last_delinq There is NULL Values
mths_since_last_record There is NULL Values
pub_rec There is NULL Values
revol_util There is NULL Values
total_acc There is NULL Values
collections_12_mths_ex_med There is NULL Values


In [8]:
# Need to fill the NA values
data['delinq_2yrs'].fillna(data['delinq_2yrs'].median(), inplace=True)
data['inq_last_6mths'].fillna(data['inq_last_6mths'].median(), inplace=True)
data['mths_since_last_delinq'].fillna(data['mths_since_last_delinq'].median(), inplace=True)
data['mths_since_last_record'].fillna(data['mths_since_last_record'].median(), inplace=True)
data['pub_rec'].fillna(data['pub_rec'].median(), inplace=True)
data['revol_util'].fillna(data['revol_util'].median(), inplace=True)
data['total_acc'].fillna(data['total_acc'].median(), inplace=True)
data['collections_12_mths_ex_med'].fillna(data['collections_12_mths_ex_med'].median(), inplace=True)

In [9]:
print(len(list(data.columns)))

25


In [10]:
# Encoding all type(object) and encoding them using LabelEncoder
for column in data.columns:
    if data[column].dtype == type(object):
        le = LabelEncoder()
        print('not able to convert', column)
        data[column] = le.fit_transform(data[column])


X = data.loc[:, data.columns != 'is_bad']
Y = data.loc[:, data.columns == 'is_bad']

logreg = LogisticRegression()

rfe = RFE(logreg, 24)
rfe = rfe.fit(X, Y.values.ravel())
print(rfe.support_)
print(rfe.ranking_)

not able to convert emp_title
not able to convert emp_length
not able to convert home_ownership
not able to convert verification_status
not able to convert pymnt_plan
not able to convert purpose_cat
not able to convert zip_code
not able to convert addr_state
not able to convert earliest_cr_line
not able to convert initial_list_status
not able to convert policy_code
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


/Users/hiuchan/anaconda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [11]:
xTrain, xTest, yTrain, yTest = train_test_split(X, Y, test_size = 0.15, random_state = 6)

In [12]:
# penalty is set to L2 for logistic regression regularization.
# cv set to five for 5 `stratified CV folds
# class_weights set to 'balance' since our dataset are very bias as I mentioned above. But I do not think it helps at all

logreg=LogisticRegressionCV(Cs=list(np.power(10.0, np.arange(0, 8)))
        ,penalty='l2'
        ,cv=5
        ,max_iter=10000
        ,solver='liblinear'
        ,class_weight='balanced'
        ,tol=0.5)

logreg.fit(xTrain, yTrain['is_bad'])

LogisticRegressionCV(Cs=[1.0, 10.0, 100.0, 1000.0, 10000.0, 100000.0, 1000000.0, 10000000.0],
           class_weight='balanced', cv=5, dual=False, fit_intercept=True,
           intercept_scaling=1.0, max_iter=10000, multi_class='warn',
           n_jobs=None, penalty='l2', random_state=None, refit=True,
           scoring=None, solver='liblinear', tol=0.5, verbose=0)

In [13]:
score = logreg.score(xTest, yTest['is_bad'])
print(score)

0.868


In [14]:
lg_probas = logreg.predict_proba(xTest)
log_loss = log_loss(yTest, lg_probas)
print('Log Lose is',log_loss)

Log Lose is 0.6623165890655648


In [15]:
predict_y = logreg.predict(xTest)
f1 = f1_score(yTest, predict_y, average="macro")
print('F1 is', f1)

F1 is 0.46466809421841543


/Users/hiuchan/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [16]:
# Create StratifiedKFold
# I use four layers network.

kfold = StratifiedKFold(n_splits=5, shuffle=True)
cvscores = []
for train, test in kfold.split(X.values, Y['is_bad'].values):
  # create model
    model = Sequential()
    model.add(Dense(10, activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(3, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    # Fit the model
    model.fit(X.values[train], Y['is_bad'].values[train], epochs=50, batch_size=30)
    # evaluate the model
    scores = model.evaluate(X.values[test], Y['is_bad'].values[test])
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

Epoch 1/50
8000/8000 [==============================] - 0s 53us/step - loss: 1.1363 - acc: 0.8425
Epoch 2/50
8000/8000 [==============================] - 0s 41us/step - loss: 0.5689 - acc: 0.8705
Epoch 3/50
8000/8000 [==============================] - 0s 42us/step - loss: 0.5140 - acc: 0.8705
Epoch 4/50
8000/8000 [==============================] - 0s 42us/step - loss: 0.4744 - acc: 0.8705
Epoch 5/50
8000/8000 [==============================] - 0s 41us/step - loss: 0.4461 - acc: 0.8705
Epoch 6/50
8000/8000 [==============================] - 0s 42us/step - loss: 0.4261 - acc: 0.8705
Epoch 7/50
8000/8000 [==============================] - 0s 42us/step - loss: 0.4122 - acc: 0.8705
Epoch 8/50
8000/8000 [==============================] - 0s 43us/step - loss: 0.4026 - acc: 0.8705
Epoch 9/50
8000/8000 [==============================] - 0s 42us/step - loss: 0.3961 - acc: 0.8705
Epoch 10/50
8000/8000 [==============================] - 0s 41us/step - loss: 0.3919 - acc: 0.8705
Epoch 11/50
8000/80

8000/8000 [==============================] - 0s 44us/step - loss: 13.8778 - acc: 0.1295
Epoch 33/50
8000/8000 [==============================] - 0s 41us/step - loss: 13.8778 - acc: 0.1295
Epoch 34/50
8000/8000 [==============================] - 0s 42us/step - loss: 13.8778 - acc: 0.1295
Epoch 35/50
8000/8000 [==============================] - 0s 42us/step - loss: 13.8778 - acc: 0.1295
Epoch 36/50
8000/8000 [==============================] - 0s 41us/step - loss: 13.8778 - acc: 0.1295
Epoch 37/50
8000/8000 [==============================] - 0s 42us/step - loss: 13.8778 - acc: 0.1295
Epoch 38/50
8000/8000 [==============================] - 0s 41us/step - loss: 13.8778 - acc: 0.1295
Epoch 39/50
8000/8000 [==============================] - 0s 42us/step - loss: 13.8778 - acc: 0.1295
Epoch 40/50
8000/8000 [==============================] - 0s 42us/step - loss: 13.8778 - acc: 0.1295
Epoch 41/50
8000/8000 [==============================] - 0s 42us/step - loss: 13.8778 - acc: 0.1295
Epoch 42/50


8000/8000 [==============================] - 0s 42us/step - loss: 2.0873 - acc: 0.8705
Epoch 14/50
8000/8000 [==============================] - 0s 41us/step - loss: 2.0873 - acc: 0.8705
Epoch 15/50
8000/8000 [==============================] - 0s 41us/step - loss: 2.0873 - acc: 0.8705
Epoch 16/50
8000/8000 [==============================] - 0s 41us/step - loss: 2.0873 - acc: 0.8705
Epoch 17/50
8000/8000 [==============================] - 0s 41us/step - loss: 2.0873 - acc: 0.8705
Epoch 18/50
8000/8000 [==============================] - 0s 40us/step - loss: 2.0873 - acc: 0.8705
Epoch 19/50
8000/8000 [==============================] - 0s 41us/step - loss: 2.0873 - acc: 0.8705
Epoch 20/50
8000/8000 [==============================] - 0s 42us/step - loss: 2.0873 - acc: 0.8705
Epoch 21/50
8000/8000 [==============================] - 0s 43us/step - loss: 2.0873 - acc: 0.8705
Epoch 22/50
8000/8000 [==============================] - 0s 42us/step - loss: 2.0873 - acc: 0.8705
Epoch 23/50
8000/8000 

8000/8000 [==============================] - 0s 44us/step - loss: 13.8778 - acc: 0.1295
Epoch 45/50
8000/8000 [==============================] - 0s 42us/step - loss: 13.8778 - acc: 0.1295
Epoch 46/50
8000/8000 [==============================] - 0s 41us/step - loss: 13.8778 - acc: 0.1295
Epoch 47/50
8000/8000 [==============================] - 0s 41us/step - loss: 13.8778 - acc: 0.1295
Epoch 48/50
8000/8000 [==============================] - 0s 41us/step - loss: 13.8778 - acc: 0.1295
Epoch 49/50
8000/8000 [==============================] - 0s 42us/step - loss: 13.8778 - acc: 0.1295
Epoch 50/50
2000/2000 [==============================] - 0s 53us/step
acc: 12.95%
57.41% (+/- 36.30%)
